In [ ]:
import os
import glob
from matplotlib import pyplot as plt
import re
import numpy as np

In [ ]:
path = r'C:\Users\parkf\Desktop\summer2021\htcChNorm' #this will be where ever you store '.out' files with channel information on MSE
showChannels = 0 #toggel to 1 if you want to see a graph for every channel, 0 if you want only subject level info

subjectList = []
ridgeVec = []
memVec = []
foldVec = []
channelDict = {}

for filename in glob.glob(os.path.join(path, '*.out')): #scan the folder to determine the size of the array we'll need
    parseThis = os.path.basename(filename)
    fileNum, parseThis = re.split('L|R|B', parseThis)
    memNum, parseThis = re.split('r', parseThis)
    memNum = re.split('p', memNum)[1]
    ridgeNum, parseThis = re.split('c', parseThis)
    foldNum = re.split('.o', parseThis)[0]
    if int(fileNum) not in subjectList:
        subjectList.append(int(fileNum))
        f = open(filename, "r")
        hold = f.read()
        channelDict[subjectList.index(int(fileNum))] = len(re.split(',', hold)) - 1
    if int(memNum) not in memVec:
        memVec.append(int(memNum))
    if float(ridgeNum) not in ridgeVec:
        ridgeVec.append(float(ridgeNum))
    if int(foldNum) not in foldVec:
        foldVec.append(int(foldNum))

for subject in range(len(subjectList)): #loop through again to actually fill out array 
    #we can't just hold everything here since some subjects will have different amounts of channels
    err = np.zeros((len(ridgeVec), len(memVec), channelDict[subject], len(foldVec)))
    for filename in glob.glob(os.path.join(path, '*.out')):
        parseThis = os.path.basename(filename)
        fileNum, parseThis = re.split('L|R|B', parseThis)
        if int(fileNum) != subjectList[subject]:
            continue
        memNum, parseThis = re.split('r', parseThis)
        memNum = re.split('p', memNum)[1]
        ridgeNum, parseThis = re.split('c', parseThis)
        foldNum = re.split('.o', parseThis)[0]
        f = open(filename, "r")
        hold = f.read()
        x = np.array(re.split(",", hold))[0:channelDict[subject]]
        for ch in range(len(x)):
            err[ridgeVec.index(float(ridgeNum)), memVec.index(int(memNum)), ch, int(foldNum)] = float(x[ch])
    errNoCv = np.mean(err, 3)
    if showChannels == 1:
        for ch in range(len(x)):
            titleString = f"Memory vs MSE, for subject {subjectList[subject]}, channel {ch}"
            plt.title(titleString)
            for i in range(len(ridgeVec)):
                plt.plot(memVec[:], errNoCv[i , :, ch], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
            plt.legend()
            plt.show()
    errNoCh = np.mean(errNoCv, 2)
    titleString = f"Memory vs MSE, for subject {subjectList[subject]}, channel's averaged"
    plt.title(titleString)
    for i in range(len(ridgeVec)):
        plt.plot(memVec[:], errNoCh[i , :], ".", label = f"lambda = {ridgeVec[i]}") ## figure out columns to pass
        plt.xlabel("Number of memory points")
        plt.ylabel("MSE")
    plt.legend()
    plt.show()
